In [12]:
import csv
import traceback
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv('/home/ubuntu/nepal_chatbot/.env')
open_ai_key = os.getenv("OPENAI_API_KEY")

try:
    if open_ai_key:
        print("OpenAI key is loaded")
    else:
        raise ValueError("OpenAI key is not set")
except Exception as e:
    print(f"Error loading OpenAI API key: {e}")

OpenAI key is loaded


In [11]:
# class ActionCaptureGrievanceText(Action):
#     def name(self) -> Text:
#         return "action_capture_grievance_text"

def load_classification_data(file_path):
    """Loads grievance classification data from CSV into a dictionary"""
    categories = []
    try:
        with open(file_path, "r", encoding="utf-8") as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                categories.append(row["Classification"].title() + " - " + row["Generic Grievance Name"].title())  # Normalize case
    except Exception as e:
        print(f"Error loading CSV file: {e}")
        traceback.print_exc()
    return list(set(categories))

cat_path = '/home/ubuntu/nepal_chatbot/resources/grievances_categorization_v1.csv'

print(load_classification_data(cat_path))

['Wildlife, Environmental - Wildlife Passage', 'Gender, Social - Gender-Based Access Issues', 'Malicious Behavior, Environmental - Fire Incidents', 'Relocation Issues - Lack Of Infrastructure Of The Resettlement Site', 'Destruction Of Agrarian Resources - Soil Pollution', 'Environmental, Social - Cutting Of Trees', 'Environmental - Noise Pollution', 'Relocation Issues - Forced Relocation Issues', 'Economic, Social - Employment Opportunities', 'Cultural, Social - Cultural Site Disturbances', 'Economic, Social - Land Acquisition Issues', 'Relocation Issues - Poor Location Of The Resettlement Site', 'Safety - Road Safety Provisions', 'Destruction Of Agrarian Resources - Crop Destruction', 'Relocation Issues - Poor Housing Quality Of The Resettlement Site', 'Destruction Of Agrarian Resources - Destruction Of Agrarian Soils', 'Relocation Of Public Utilities - Access To Electricity', 'Malicious Behavior - Theft Of Tools Or Equipment', 'Environmental - Drainage And Sewage Management', 'Wildli

In [20]:
# def run(self, dispatcher: CollectingDispatcher, tracker: Tracker, domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:
#     grievance_details = tracker.latest_message.get("text")
#     print(f"User input: {grievance_details}")

def open_ai_categorization(grievance_details, csv_file_path):
    # Load CSV Data
    # csv_file_path = "/home/ubuntu/nepal_chatbot/resources/grievances_categorization_v1.csv"
    classification_data = load_classification_data(csv_file_path)


    # Step 1: use OpenAI but restrict the category choices
    predefined_categories = load_classification_data(csv_file_path)# Extract unique categories
    category_list_str = "\n".join(f"- {c}" for c in predefined_categories)  # Format as list

    client = OpenAI(api_key=open_ai_key)

    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an assistant helping to categorize grievances."},
            {"role": "user", "content": f"""
                Categorize this grievance: "{grievance_details}"
                Only choose from the following categories:
                {category_list_str}
                Do not create new categories.
                Reply only with the categories, if many categories apply just list them with the following format:
                1: category, 2: category, 3: category etc when applicable
            """}
        ],
        model="gpt-4",
    )

    best_category = response.choices[0].message.content.strip()
    return(best_category)

    # # Step 2: Validate category with the user
    # buttons = [
    #     {"title": "Yes", "payload": "/agree"},
    #     {"title": "No, choose another category", "payload": "/deny"},
    #     {"title": "Exit", "payload": "/exit_grievance_process"}
    # ]

    # dispatcher.utter_message(
    #     text=f"Here's the category I identified: '{best_category}'. Does this seem correct?",
    #     buttons=buttons
    # )

    # return [
    #     SlotSet("grievance_details", grievance_details),
    #     SlotSet("grievance_category", best_category)
    # ]
grievance = "tehy killed my duck and stole my rice"

print(open_ai_categorization(grievance, cat_path))

1: Wildlife, Environmental - Wildlife Destruction, 2: Malicious Behavior - Theft Of Crops
